# Static elastic constants
This Notebook follows the workflow for setting up, running, and processing the elastic_constants_static calculations.

**Quick Notes:**

- All input scripts take key-value pairs where the "key"s correspond to the calculation's input parameters.

- Multiple calculations are prepared by specifying multiple values for the same key (on separate lines).

- The special "buildcombos" key accesses predefined functions for generating lists of input parameter values for certain sets of keys.  See documentation for more details.

__Global workflow details:__

This Notebook requires calculation_crystal_space_group records and the "unique_crystals.csv" file, both of which are generated by the "3. Crystal relaxation" Notebook.

The calculation_elastic_constants_static records generated are used as inputs for dislocation creation Notebooks.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import os
from datetime import date
from math import floor
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://bokeh.pydata.org/
import bokeh
import bokeh.plotting
import bokeh.resources
import bokeh.io
import bokeh.models

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('demo')

### Load unique crystals

In [3]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

104


### Limit unique crystals to include (optional) 

In [4]:
#unique_crystals = unique_crystals[unique_crystals.potential_LAMMPS_id == '2012--Park-H--Mo--LAMMPS--ipr1']
#unique_crystals = unique_crystals[unique_crystals.composition == '']
#unique_crystals = unique_crystals[unique_crystals.prototype == '']
print(len(unique_crystals))

104


## 1. Run elastic_constants_static calculations

This calculation evaluates the elastic constants for a crystal using small strain deformations.

In [5]:
calculation = iprPy.load_calculation('elastic_constants_static')
run_directory = iprPy.load_run_directory('demo_1')

### Show calculation's allowed keys

These are the keys that the input script will accept.

In [6]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'strainrange', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


### Write input script

In [7]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information from crystal_space_group results
buildcombos                 atomicparent load_file parent
parent_record               calculation_crystal_space_group
parent_load_key             unit-cell-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   5 5 5

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
strainrange                  1e-5
strainrange                  1e-6
strainrange                  1e-7
strainrange                  1e-8
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

### Parse input script

In [8]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_name list using unique crystals

In [9]:
input_dict['parent_name'] = unique_crystals.calc_key.tolist()

### Prepare calculations

In [10]:
database.prepare(run_directory, calculation, **input_dict)

In [11]:
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 416 of style calculation_elastic_constants_static
 - 0 are complete
 - 416 still to run
 - 0 issued errors


### Run calculations

In [12]:
database.runner(run_directory)

Runner started with pid 13716
No simulations left to run


In [13]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
errors = []
for error in error_df.error:
    lines = error.splitlines()
    err = ''
    for i in range(len(lines)-1, -1, -1):
        if 'Error:' in lines[i]:
            err = '\n'.join(lines[i:-1])
            break
        if i == 0:
            err = error
    errors.append(err)
for error in np.unique(errors):
    print(error)

6 calculations issued errors:
AssertionError: Cij values not valid
ValueError: Invalid LAMMPS input: 
Last command: read_restart initial.restart


## 2. Calculation analysis

### Retrieve finished calculation results

In [14]:
records = database.get_records_df(style='calculation_elastic_constants_static', full=True, flat=False, status='finished')
pot_records = database.get_records_df(style='potential_LAMMPS')

### Identify compositions

In [15]:
iprPy.analysis.assign_composition(records, database)

In [16]:
# Settings
outputpath = 'C:/Users/lmh1/Documents/demo_results'
#savecolumns = ['calc_key', 'composition', 'prototype', 'family', 'a', 
#               'strainrange', '', 'gamma_fs']

### Save raw data

In [17]:
records

,C,LAMMPS_version,error,family,iprPy_version,key,load_file,load_options,load_style,potential_LAMMPS_id,...,potential_id,potential_key,raw_cij_negative,raw_cij_positive,script,sizemults,status,strainrange,symbols,composition
0,[[ 2.01919471 -0.01142724 0.40949518 0. ...,22 Aug 2018,NaN,oqmd-1214728,0.8.3,0010db6d-43e6-4cce-a346-84ca7f804268,b7937265-1019-453a-a29a-f92ba7d19960.json,key unit-cell-atomic-system,system_model,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,"[[2.0192659466969354, -0.011485207998714051, 0...","[[2.019123475258122, -0.011369273835402925, 0....",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-05,[Ni],Ni
1,[[ 2.77109939e+00 1.05794816e+00 1.24983931e...,22 Aug 2018,NaN,oqmd-1220003,0.8.3,00821bb4-63f6-4f87-a98b-eb2bd367d2cc,a2bfffbd-606e-4903-b81b-d15a4016e2e0.json,key unit-cell-atomic-system,system_model,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,"[[2.7657894308141806, 0.8416525559828969, 1.23...","[[2.7764093524229594, 1.0948774553654061, 1.05...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-08,"[Cu, Ni]",CuNi
2,[[0.30255725 0.22901108 0.36334093 0. ...,26 Jan 2017-ICMS,NaN,A3'--alpha-La--double-hcp,0.8.3,011f262b-22f6-4f12-a6a5-f982b855df1e,475b69d4-6fc9-4b37-bcc7-50155b565579.json,key unit-cell-atomic-system,system_model,2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3,...,2003--Mendelev-M-I-Han-S-Srolovitz-D-J-et-al--...,06bf7ccd-ea6a-4744-bfbc-adefa5bcdd60,"[[0.30255623035642776, 0.22901092667016384, 0....","[[0.30255826334892755, 0.22901123792907924, 0....",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-07,[Fe],Fe
3,[[ 6.46155092e-01 6.53178728e-01 6.53178727e...,26 Jan 2017-ICMS,NaN,D0_3--BiF3,0.8.3,019833ce-5af3-4938-9db2-00bc7d41713c,dd2a5d41-9e9d-4788-88ea-472676de8229.json,key unit-cell-atomic-system,system_model,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,"[[0.6461553468768753, 0.6531788652307763, 0.65...","[[0.6461548366974301, 0.6531785910917108, 0.65...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-07,"[Cu, Ni]",CuNi3
4,[[ 4.66734740e-01 4.72271563e-01 3.76785109e...,22 Aug 2018,NaN,oqmd-1215683,0.8.3,0277e0bd-8e45-429d-a2f1-3c7a63dc4380,f5ebaa16-068a-43d1-8910-51df466d4203.json,key unit-cell-atomic-system,system_model,2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3,...,2003--Mendelev-M-I-Han-S-Srolovitz-D-J-et-al--...,06bf7ccd-ea6a-4744-bfbc-adefa5bcdd60,"[[0.46673464997427583, 0.4722715593686777, 0.3...","[[0.46673482957211243, 0.47227156734872644, 0....",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-08,[Fe],Fe
5,[[0.83104393 0.88640811 0.85295701 0. ...,22 Aug 2018,NaN,mp-1010136,0.8.3,02de4e04-2264-4868-be76-58d39da13a6d,76fe0d14-43ef-4061-bb91-b150bed4579b.json,key unit-cell-atomic-system,system_model,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,...,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,"[[0.8310720716013452, 0.886429093749294, 0.852...","[[0.8310157874422673, 0.886387116300252, 0.852...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-05,[Cu],Cu
6,[[-0.47048672 -0.10663233 0.49588942 -0.35094...,22 Aug 2018,NaN,A7--alpha-As,0.8.3,02fa16e2-99a2-43de-a13d-63105268f27b,9e03d38d-d121-4a04-96b3-3d340d330cce.json,key unit-cell-atomic-system,system_model,2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3,...,2003--Mendelev-M-I-Han-S-Srolovitz-D-J-et-al--...,06bf7ccd-ea6a-4744-bfbc-adefa5bcdd60,"[[-0.4704874833477701, -0.10663241109543087, 0...","[[-0.47048596046872504, -0.10663224254969794, ...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-07,[Fe],Fe
7,[[ 1.72815626 0.11631619 0.11631619 0. ...,22 Aug 2018,NaN,Ah--alpha-Po--sc,0.8.3,05042971-6aa4-4f20-9738-b7650f0ad32b,cf1926b8-9d1e-4275-8e07-23f82d4380c2.json,key unit-cell-atom

In [18]:
data = []
for record in records.itertuples():
    crystal = unique_crystals[unique_crystals.calc_key == record.load_file.split('.')[0]].iloc[0]
    
    dat = {}
    dat['calc_key'] = record.key
    dat['potential_LAMMPS_key'] = crystal.potential_LAMMPS_key
    dat['potential_LAMMPS_id'] = crystal.potential_LAMMPS_id
    dat['potential_key'] = crystal.potential_key
    dat['potential_id'] = crystal.potential_id
    dat['prototype'] = crystal.prototype
    dat['family'] = crystal.family
    dat['composition'] = record.composition
    dat['strainrange'] = record.strainrange
    dat['a'] = crystal.a
    dat['b'] = crystal.b
    dat['c'] = crystal.c
    dat['alpha'] = crystal.alpha
    dat['beta'] = crystal.beta
    dat['gamma'] = crystal.gamma

    dat['straindirection'] = 'positive'
    cij = uc.get_in_units(record.raw_cij_positive, 'GPa')
    dat['C11'] = '%.3f' %cij[0,0]
    dat['C12'] = '%.3f' %cij[0,1]
    dat['C13'] = '%.3f' %cij[0,2]
    dat['C14'] = '%.3f' %cij[0,3]
    dat['C15'] = '%.3f' %cij[0,4]
    dat['C16'] = '%.3f' %cij[0,5]
    dat['C21'] = '%.3f' %cij[1,0]
    dat['C22'] = '%.3f' %cij[1,1]
    dat['C23'] = '%.3f' %cij[1,2]
    dat['C24'] = '%.3f' %cij[1,3]
    dat['C25'] = '%.3f' %cij[1,4]
    dat['C26'] = '%.3f' %cij[1,5]
    dat['C31'] = '%.3f' %cij[2,0]
    dat['C32'] = '%.3f' %cij[2,1]
    dat['C33'] = '%.3f' %cij[2,2]
    dat['C34'] = '%.3f' %cij[2,3]
    dat['C35'] = '%.3f' %cij[2,4]
    dat['C36'] = '%.3f' %cij[2,5]
    dat['C41'] = '%.3f' %cij[3,0]
    dat['C42'] = '%.3f' %cij[3,1]
    dat['C43'] = '%.3f' %cij[3,2]
    dat['C44'] = '%.3f' %cij[3,3]
    dat['C45'] = '%.3f' %cij[3,4]
    dat['C46'] = '%.3f' %cij[3,5]
    dat['C51'] = '%.3f' %cij[4,0]
    dat['C52'] = '%.3f' %cij[4,1]
    dat['C53'] = '%.3f' %cij[4,2]
    dat['C54'] = '%.3f' %cij[4,3]
    dat['C55'] = '%.3f' %cij[4,4]
    dat['C56'] = '%.3f' %cij[4,5]
    dat['C61'] = '%.3f' %cij[5,0]
    dat['C62'] = '%.3f' %cij[5,1]
    dat['C63'] = '%.3f' %cij[5,2]
    dat['C64'] = '%.3f' %cij[5,3]
    dat['C65'] = '%.3f' %cij[5,4]
    dat['C66'] = '%.3f' %cij[5,5]
    data.append(deepcopy(dat))

    dat['straindirection'] = 'negative'
    cij = uc.get_in_units(record.raw_cij_negative, 'GPa')
    dat['C11'] = '%.3f' %cij[0,0]
    dat['C12'] = '%.3f' %cij[0,1]
    dat['C13'] = '%.3f' %cij[0,2]
    dat['C14'] = '%.3f' %cij[0,3]
    dat['C15'] = '%.3f' %cij[0,4]
    dat['C16'] = '%.3f' %cij[0,5]
    dat['C21'] = '%.3f' %cij[1,0]
    dat['C22'] = '%.3f' %cij[1,1]
    dat['C23'] = '%.3f' %cij[1,2]
    dat['C24'] = '%.3f' %cij[1,3]
    dat['C25'] = '%.3f' %cij[1,4]
    dat['C26'] = '%.3f' %cij[1,5]
    dat['C31'] = '%.3f' %cij[2,0]
    dat['C32'] = '%.3f' %cij[2,1]
    dat['C33'] = '%.3f' %cij[2,2]
    dat['C34'] = '%.3f' %cij[2,3]
    dat['C35'] = '%.3f' %cij[2,4]
    dat['C36'] = '%.3f' %cij[2,5]
    dat['C41'] = '%.3f' %cij[3,0]
    dat['C42'] = '%.3f' %cij[3,1]
    dat['C43'] = '%.3f' %cij[3,2]
    dat['C44'] = '%.3f' %cij[3,3]
    dat['C45'] = '%.3f' %cij[3,4]
    dat['C46'] = '%.3f' %cij[3,5]
    dat['C51'] = '%.3f' %cij[4,0]
    dat['C52'] = '%.3f' %cij[4,1]
    dat['C53'] = '%.3f' %cij[4,2]
    dat['C54'] = '%.3f' %cij[4,3]
    dat['C55'] = '%.3f' %cij[4,4]
    dat['C56'] = '%.3f' %cij[4,5]
    dat['C61'] = '%.3f' %cij[5,0]
    dat['C62'] = '%.3f' %cij[5,1]
    dat['C63'] = '%.3f' %cij[5,2]
    dat['C64'] = '%.3f' %cij[5,3]
    dat['C65'] = '%.3f' %cij[5,4]
    dat['C66'] = '%.3f' %cij[5,5]
    data.append(deepcopy(dat))

columns = ['calc_key', 'potential_LAMMPS_key', 'potential_LAMMPS_id', 'potential_key', 'potential_id',
           'composition', 'prototype', 'family', 
           'a', 'b', 'c', 'alpha', 'beta', 'gamma',
           'strainrange', 'straindirection', 
           'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
           'C21', 'C22', 'C23', 'C24', 'C25', 'C26',
           'C31', 'C32', 'C33', 'C34', 'C35', 'C36',
           'C41', 'C42', 'C43', 'C44', 'C45', 'C46',
           'C51', 'C52', 'C53', 'C54', 'C55', 'C56',
           'C61', 'C62', 'C63', 'C64', 'C65', 'C66']

data = pd.DataFrame(data, columns=columns).sort_values(['potential_LAMMPS_id', 'composition', 'prototype', 'strainrange', 'straindirection'])

In [19]:
data.to_csv('elastic-constants.csv')

### Save raw crystal-specific data

In [20]:
for implememtation_key in np.unique(data.potential_LAMMPS_key):
    imp_results = data[data.potential_LAMMPS_key == implememtation_key]
    potential = imp_results.iloc[0].potential_id
    implementation = imp_results.iloc[0].potential_LAMMPS_id
    
    contentpath = os.path.join(outputpath, potential, implementation)
    if not os.path.isdir(contentpath):
        os.makedirs(contentpath)
    
    for composition in np.unique(imp_results.composition):
        comp_results = imp_results[imp_results.composition == composition].sort_values(['family', 'a', 'strainrange', 'straindirection'])
        fstem = 'elastic.' + composition
        
        comp_results.to_csv(os.path.join(contentpath, fstem + '.csv'), index=False)


### Add info to PotentialProperties records

This is for generating XML records that the Interatomic Potential Repository uses to automatically build webcontent (done elsewhere).

In [21]:
# Add calculation data to PotentialProperties records
for implememtation_key in np.unique(data.potential_LAMMPS_key):
    imp_records = data[data.potential_LAMMPS_key == implememtation_key].sort_values('family')
    pot_record = pot_records[pot_records.key == implememtation_key].iloc[0]
    implementation_id = pot_record.id
    potential_id = pot_record.pot_id
    potential_key = pot_record.pot_key
    
    record_name = 'properties.' + implementation_id
    try:
        record = database.get_record(name=record_name, style='PotentialProperties')
    except:
        new = True
        content = DM()
        content['per-potential-properties'] = DM()
        content['per-potential-properties']['potential'] = DM()
        content['per-potential-properties']['potential']['key'] = potential_key
        content['per-potential-properties']['potential']['id'] = potential_id
        content['per-potential-properties']['implementation'] = DM()
        content['per-potential-properties']['implementation']['key'] = implememtation_key
        content['per-potential-properties']['implementation']['id'] = implementation_id
    else:
        content = DM(record.content)
        new = False
    
    content['per-potential-properties']['elastic-constants'] = ec_model = DM()
    
    for composition in np.unique(imp_records.composition):
        comp_records = imp_records[imp_records.composition == composition]
        comp_model = DM()
        comp_model['composition'] = composition
        
        for prototype in np.unique(comp_records.prototype):
            proto_records = comp_records[comp_records.prototype == prototype]
            proto_model = DM()
            proto_model['prototype'] = prototype
            
            for alat in np.unique(proto_records.a):
                alat_records = proto_records[proto_records.a == alat]
                alat_model = DM()
                alat_model['a'] = alat
                
                for series in alat_records.sort_values(['strainrange', 'straindirection']).itertuples():
                    measurement = DM()
                    if series.straindirection == 'positive':
                        measurement['strain'] = '+%s' %series.strainrange
                    elif series.straindirection == 'negative':
                        measurement['strain'] = '-%s' %series.strainrange
                    else:
                        raise ValueError('Unknown straindirection')
                    measurement['C11'] = series.C11
                    measurement['C12'] = series.C12
                    measurement['C13'] = series.C13
                    measurement['C14'] = series.C14
                    measurement['C15'] = series.C15
                    measurement['C16'] = series.C16
                    measurement['C21'] = series.C21
                    measurement['C22'] = series.C22
                    measurement['C23'] = series.C23
                    measurement['C24'] = series.C24
                    measurement['C25'] = series.C25
                    measurement['C26'] = series.C26
                    measurement['C31'] = series.C31
                    measurement['C32'] = series.C32
                    measurement['C33'] = series.C33
                    measurement['C34'] = series.C34
                    measurement['C35'] = series.C35
                    measurement['C36'] = series.C36
                    measurement['C41'] = series.C41
                    measurement['C42'] = series.C42
                    measurement['C43'] = series.C43
                    measurement['C44'] = series.C44
                    measurement['C45'] = series.C45
                    measurement['C46'] = series.C46
                    measurement['C51'] = series.C51
                    measurement['C52'] = series.C52
                    measurement['C53'] = series.C53
                    measurement['C54'] = series.C54
                    measurement['C55'] = series.C55
                    measurement['C56'] = series.C56
                    measurement['C61'] = series.C61
                    measurement['C62'] = series.C62
                    measurement['C63'] = series.C63
                    measurement['C64'] = series.C64
                    measurement['C65'] = series.C65
                    measurement['C66'] = series.C66
                    
                    alat_model.append('measurement', measurement)
                proto_model.append('alats', alat_model)
            comp_model.append('prototypes', proto_model)
        ec_model.append('compositions', comp_model)
    
    if new:
        database.add_record(name=record_name, style='PotentialProperties', content=content.xml())
    else:
        database.update_record(record=record, content=content.xml())